# Implementing a Viterbi Decoder and Evaluation for Sequence Labeling

In this assignment, you will build a Viterbi decoder for an LSTM named-entity recognition model. As we mentioned in class, recurrent and bidirectional recurrent neural networks, of which LSTMs are the most common examples, can be used to perform sequence labeling. Although these models encode information from the surrounding words in order to make predictions, there are no "hard" constraints on what tags can appear where.

There hard constraints are particularly important for tasks that label spans of more than one token. The most common example of a span-labeling task is named-entity recognition (NER). As described in Eisenstein, Jurafksy & Martin, and other texts, the goal of NER is to label spans of one or more words as _mentions_ of an _entity_, such as a person, location, organization, etc.

The most common approach to NER is to reduce it to a sequence-labeling task, where each token in the input is labeled either with an `O`, if it is "outside" any named-entity span, or with `B-TYPE`, if it is the first token in an entity of type `TYPE`, or with `I-TYPE`, if it is the second or later token in an entity of type `TYPE`. Distinguishing between the first and later tokens of an entity allow us to identify distinct entity spans even when they are adjacent.

Common values of `TYPE` include `PER` for person, `LOC` for location, `DATE` for date, and so on. In the dataset we load below, there are 17 distinct types.

The span-labeling scheme just described implies that the labels on tokens must obey certain constraints: the tag `I-PER` must follow either `B-PER` or another `I-PER`. It cannot follow `O`, `B-LOC`, or `I-LOC`, i.e., a tag for a different entity type. By themselves, LSTMs or bidirectional LSTMs cannot directly enforce these constraints. This is one reason why conditional random fields (CRFs), which _can_ enforce these constraints, are often layered on top of these recurrent models.

In this assignment, you will implement the simplest possible CRF: a CRF so simple that it does not require any training. Rather, it will assign weight 1 to any sequence of tags that obeys the constraints and weight 0 to any sequence of tags that violates them. The inputs to the CRF, which are analogous to the emission probabilities in an HMM, will come from an LSTM.

But first, in order to test your decoder, you will also implement some functions to evaluate the output of an NER system according to two metrics:
1. You will count the number of _violations_ of the NER label constraints, i.e., how many times `I-TYPE` follows `O` or a tag of a different type or occurs at the beginning of a sentence. This number will be greater than 0 in the raw LSTM output, but should be 0 for your CRF output.
1. You will compute the _span-level_ precision, recall, and F1 of NER output. Although the baseline LSTM was trained to achieve high _token-level_ accuracy, this metric can be misleadingly high, since so many tokens are correctly labeled `O`. In other words, what proportion of spans predicted by the model line up exactly with spans in the gold standard, and what proportion of spans in the gold standard were predicted by the model? For more, see the original task definition: https://www.aclweb.org/anthology/W03-0419/.

We start with loading some code and data and the describe your tasks in more detail.

## Set Up Dependencies and Definitions

In [1]:
!pip install --upgrade spacy allennlp
import spacy
print(spacy.__version__)
!pip install seqeval
import numpy as np
from keras.callbacks import Callback
from seqeval.metrics import f1_score, classification_report
from seqeval.metrics import accuracy_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score
from seqeval.metrics import recall_score
from seqeval.metrics import precision_score

Requirement already up-to-date: spacy in /usr/local/lib/python3.6/dist-packages (2.2.4)
Requirement already up-to-date: allennlp in /usr/local/lib/python3.6/dist-packages (0.9.0)
2.2.4


Using TensorFlow backend.


In [2]:
from typing import Iterator, List, Dict
import torch
import torch.optim as optim
import numpy as np
from allennlp.data import Instance
from allennlp.data.fields import TextField, SequenceLabelField
from allennlp.data.dataset_readers import DatasetReader
from allennlp.common.file_utils import cached_path
from allennlp.data.token_indexers import TokenIndexer, SingleIdTokenIndexer
from allennlp.data.tokenizers import Token
from allennlp.data.vocabulary import Vocabulary
from allennlp.models import Model
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.training.metrics import CategoricalAccuracy
from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer
from allennlp.predictors import SentenceTaggerPredictor
from allennlp.data.dataset_readers import conll2003

torch.manual_seed(1)

In [0]:
class LstmTagger(Model):
  def __init__(self,
               word_embeddings: TextFieldEmbedder,
               encoder: Seq2SeqEncoder,
               vocab: Vocabulary) -> None:
    super().__init__(vocab)
    self.word_embeddings = word_embeddings
    self.encoder = encoder
    self.hidden2tag = torch.nn.Linear(in_features=encoder.get_output_dim(),
                                      out_features=vocab.get_vocab_size('labels'))
    self.accuracy = CategoricalAccuracy()

  def forward(self,
              tokens: Dict[str, torch.Tensor],
              metadata,
              tags: torch.Tensor = None) -> Dict[str, torch.Tensor]:
    mask = get_text_field_mask(tokens)
    embeddings = self.word_embeddings(tokens)
    encoder_out = self.encoder(embeddings, mask)
    tag_logits = self.hidden2tag(encoder_out)
    output = {"tag_logits": tag_logits}
    if tags is not None:
      self.accuracy(tag_logits, tags, mask)
      output["loss"] = sequence_cross_entropy_with_logits(tag_logits, tags, mask)

    return output

  def get_metrics(self, reset: bool = False) -> Dict[str, float]:
    return {"accuracy": self.accuracy.get_metric(reset)}

## Import Data

In [4]:
reader = conll2003.Conll2003DatasetReader()
train_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.train.ner.sample'))
validation_dataset = reader.read(cached_path('http://www.ccs.neu.edu/home/dasmith/onto.development.ner.sample'))

vocab = Vocabulary.from_instances(train_dataset + validation_dataset)

562it [00:00, 21197.26it/s]
23it [00:00, 14526.27it/s]
100%|██████████| 585/585 [00:00<00:00, 71927.65it/s]


## Define and Train Model

In [5]:
EMBEDDING_DIM = 6
HIDDEN_DIM = 6
token_embedding = Embedding(num_embeddings=vocab.get_vocab_size('tokens'),
                            embedding_dim=EMBEDDING_DIM)
word_embeddings = BasicTextFieldEmbedder({"tokens": token_embedding})
lstm = PytorchSeq2SeqWrapper(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, bidirectional=False, batch_first=True))
model = LstmTagger(word_embeddings, lstm, vocab)
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
# optimizer = optim.AdamW(model.parameters(), lr=1e-4, eps=1e-8)
optimizer = optim.SGD(model.parameters(), lr=0.1)
iterator = BucketIterator(batch_size=2, sorting_keys=[("tokens", "num_tokens")])
iterator.index_with(vocab)
trainer = Trainer(model=model,
                  optimizer=optimizer,
                  iterator=iterator,
                  train_dataset=train_dataset,
                  validation_dataset=validation_dataset,
                  patience=10,
                  num_epochs=100,
                  cuda_device=cuda_device)
trainer.train()

accuracy: 0.8442, loss: 0.9156 ||: 100%|██████████| 281/281 [00:01<00:00, 246.55it/s]
accuracy: 0.7878, loss: 1.1969 ||: 100%|██████████| 12/12 [00:00<00:00, 410.83it/s]
accuracy: 0.8442, loss: 0.7295 ||: 100%|██████████| 281/281 [00:01<00:00, 249.57it/s]
accuracy: 0.7878, loss: 1.1977 ||: 100%|██████████| 12/12 [00:00<00:00, 474.81it/s]
accuracy: 0.8442, loss: 0.7157 ||: 100%|██████████| 281/281 [00:01<00:00, 251.27it/s]
accuracy: 0.7878, loss: 1.1781 ||: 100%|██████████| 12/12 [00:00<00:00, 424.98it/s]
accuracy: 0.8442, loss: 0.7068 ||: 100%|██████████| 281/281 [00:01<00:00, 251.58it/s]
accuracy: 0.7878, loss: 1.1627 ||: 100%|██████████| 12/12 [00:00<00:00, 476.29it/s]
accuracy: 0.8442, loss: 0.6987 ||: 100%|██████████| 281/281 [00:01<00:00, 248.27it/s]
accuracy: 0.7878, loss: 1.1742 ||: 100%|██████████| 12/12 [00:00<00:00, 461.69it/s]
accuracy: 0.8442, loss: 0.6910 ||: 100%|██████████| 281/281 [00:01<00:00, 248.72it/s]
accuracy: 0.7878, loss: 1.1608 ||: 100%|██████████| 12/12 [00:00

{'best_epoch': 99,
 'best_validation_accuracy': 0.8836734693877552,
 'best_validation_loss': 0.382393110737515,
 'epoch': 99,
 'peak_cpu_memory_MB': 2799.88,
 'peak_gpu_0_memory_MB': 541,
 'training_accuracy': 0.9226776824488931,
 'training_cpu_memory_MB': 2799.88,
 'training_duration': '0:02:01.403297',
 'training_epochs': 99,
 'training_gpu_0_memory_MB': 541,
 'training_loss': 0.2021263821864189,
 'training_start_epoch': 0,
 'validation_accuracy': 0.8836734693877552,
 'validation_loss': 0.382393110737515}

## Evaluation

The simple code below creators a `predictor` object that applies the model to an input example and then loops over the examples in the validation set, printing out the input token, gold-standard output, and model output. You can see from these methods how to access data and model outputs for evaluation.

In [0]:
predictor = SentenceTaggerPredictor(model, dataset_reader=reader)

def tag_sentence(s):
  tag_ids = np.argmax(predictor.predict_instance(s)['tag_logits'], axis=-1)
  a.append(predictor.predict_instance(s)['tag_logits'])
  fields = zip(s['tokens'], s['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
  return list(fields)

a = []
baseline_output= [tag_sentence(i) for i in validation_dataset]

Now, you can implement two evaluation functions: `violations` and `span_stats`.

In [0]:
def compare_violations(tag1, tag2):
    
    if tag1[0] == 'B' and tag2[0] == 'I':
        if tag1[2:] == tag2[2:]:
            return 0
        else:
            return 1
    if tag1[0] == 'I' and tag2[0] == 'I':
        if tag1[2:] == tag2[2:]:
            return 0
        else:
            return 1
    if tag1[0] == 'I' and tag2[0] == 'B':
        return 0
    if tag1[0] == 'B' and tag2[0] == 'B':
        return 0
    else:
        return 0

In [8]:
# TODO: count the number of NER label violations
# such as O followed by I-TYPE or B-TYPE followed by
# I-OTHER_TYPE
# Take tagger output as input

def violations(tagged):
    count = 0
    violations = []
    for i in range(0, len(tagged)):
        for j in range(0, len(tagged[i])):
            violations.append(tagged[i][j][2])
    i = 0
    j = 1
    while i < len(violations) - 1:
        while j < len(violations):
            if violations[i] == 'O':
                if (violations[j][0] == 'I'):
                    count = count + 1
            else:
                if (compare_violations(violations[i], violations[j])) == 1:
                    count = count + 1
            i = i + 1
            j = j + 1
    return count

# TODO: return the span-level precision, recall, and F1
# Take tagger output as input
def span_stats(tagged):
  
  gold = []
  pred = []
  name = []
  for i in range(0,len(tagged)):
    temp_gold = []
    temp_pred = []
    temp_name = []
    for j in range(0,len(tagged[i])):
      temp_gold.append(tagged[i][j][1])
      temp_pred.append(tagged[i][j][2])
      temp_name.append(tagged[i][j][0])

    gold.append(temp_gold)
    pred.append(temp_pred)
    name.append(temp_name)
    precision = precision_score(gold, pred)
    recall = recall_score(gold, pred)
    f1 = 2 * (precision * recall) / (precision + recall)

  return {'precision': precision,
          'recall': recall,
          'f1': f1}

## You can check how many violations are made by the model output in predictor.
print("The model output in Predictor :")
print("Violations ~")
print(violations(baseline_output))
print("Span Statistics ~")
print(span_stats(baseline_output))

The model output in Predictor :
Violations ~
33
Span Statistics ~
{'precision': 0.171875, 'recall': 0.2558139534883721, 'f1': 0.205607476635514}


## Decoding

Now you can finally implement the simple Viterbi decoder. The `predictor` object, when applied to an input sentence, first calculates the scores for each possible output tag for each token. See the line `predictor.predict_instance(i)['tag_logits']` in the code above.

Then, you will construct a transition matrix. You can use the code below to get a list of the tags the model knows about. For a set of K tags, construct a K-by-K matrix with a log(1)=0 when a transition between a given tag pair is valid and a log(0)=-infinity otherwise.

Finally, implement a Viterbi decoder that takes the predictor object and a dataset object and outputs tagged data, just like the `tag_sentence` function above. It should use the Viterbi algorithm with the (max, plus) semiring. You'll be working with sums of log probabilities instead of products of probabilties.

Run your `violations` function on the output of this decoder to make sure that there are no invalid tag transitions. Also, compare the span-level metrics on `baseline_output` and your new output using your `span_stats` function.

In [9]:
# This code show how to map from output vector components to labels
vocabulary = vocab.get_index_to_token_vocabulary('labels')
print(vocabulary)

{0: 'O', 1: 'B-GPE', 2: 'I-ORG', 3: 'I-DATE', 4: 'B-CARDINAL', 5: 'I-EVENT', 6: 'B-PERSON', 7: 'B-NORP', 8: 'B-DATE', 9: 'B-ORG', 10: 'B-LOC', 11: 'I-LOC', 12: 'I-FAC', 13: 'I-PERSON', 14: 'I-GPE', 15: 'I-CARDINAL', 16: 'B-EVENT', 17: 'I-TIME', 18: 'I-WORK_OF_ART', 19: 'B-ORDINAL', 20: 'B-FAC', 21: 'B-TIME', 22: 'I-LAW', 23: 'I-QUANTITY', 24: 'I-NORP', 25: 'I-MONEY', 26: 'B-MONEY', 27: 'B-WORK_OF_ART', 28: 'B-QUANTITY', 29: 'B-LAW', 30: 'B-PRODUCT', 31: 'I-PRODUCT', 32: 'B-PERCENT', 33: 'I-PERCENT'}


In [10]:
# The transition matrix M X M is created , M is of size vocab, here 34.
# The log(0/1) values are assigned based on the violations we discussed above.
vocab_size = len(vocabulary)
kxkmatrix = np.zeros((vocab_size, vocab_size))
for i in range(0, vocab_size):
  for j in range(0, vocab_size):
    if(vocabulary[i] == 'O' and vocabulary[j] == 'O'):
      kxkmatrix[i][j] = np.log(1)
    elif vocabulary[i] == 'O':
      if vocabulary[j][0] == 'I':
        kxkmatrix[i][j] = np.log(0)
      else:
        kxkmatrix[i][j] = np.log(1)
    else :
      if (compare_violations(vocabulary[i], vocabulary[j]) == 1):
        kxkmatrix[i][j] = np.log(0)
      else:
        kxkmatrix[i][j] = np.log(1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: divide by zero encountered in log
  


In [0]:
# We are using max,plus semininrg
def viterbi(s):
    emission = predictor.predict_instance(s)['tag_logits']
    backpointers = np.zeros((len(emission) - 1, vocab_size))
    viterbi = emission
    
    for i in range(1, len(emission)):
        for j in range(0, vocab_size):
            kxkmatrix_max = np.max([kxkmatrix[k][j] + viterbi[i - 1][k] for k in range(vocab_size)])
            viterbi[i][j] = emission[i][j] + kxkmatrix_max
            backpointers[i - 1][j] = np.argmax([kxkmatrix[k][j] + viterbi[i - 1][k] for k in range(vocab_size)])

    tag_ids = backtrack_path(viterbi, backpointers, emission)
    fields_new = zip(s['tokens'], s['tags'], [model.vocab.get_token_from_index(i, 'labels') for i in tag_ids])
    return list(fields_new)

def backtrack_path(viterbi, backpointers, emission):
    next_pointer = np.argmax([viterbi[len(emission) - 1][k] for k in range(vocab_size)])
    tag_ids = np.zeros(len(emission))
    tag_ids[len(emission) - 1] = next_pointer
    i = len(emission) - 1
    while i > -1:
      #print(i)
      i = i - 1
      next_pointer = backpointers[i][int(next_pointer)]
      tag_ids[i] = next_pointer
    return tag_ids



In [12]:
print("The model output in Viterbi Decoder :")
#viterbi_output = [viterbi(i, model, predictor, kxkmatrix) for i in validation_dataset]
viterbi_output = [viterbi(i) for i in validation_dataset]
print("Violations ~")
print(violations(viterbi_output))
print("Span Statistics ~")
print(span_stats(viterbi_output))

The model output in Viterbi Decoder :
Violations ~
0
Span Statistics ~
{'precision': 0.32608695652173914, 'recall': 0.3488372093023256, 'f1': 0.33707865168539325}
